<a href="https://colab.research.google.com/github/Sunipowa/chatbot_rule-based/blob/main/chatbot_theo_rule_based_%2B_giao_di%E1%BB%87n_b%E1%BA%B1ng_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.5 MB/s eta 0:00:00


In [38]:
def reset_state():
    return {"step": 0, "people": None, "time": None, "name": None, "phone": None}


def chatbot_step(user_input, state):
    user_input_clean = user_input.lower().strip()
    step = state["step"]
    response = ""

    if step == 0:
        if any(kw in user_input_clean for kw in ["xin chào", "hello", "cửa hàng", "bạn ơi"]):
            response = (
                "👋 Kính chào quý khách! Mình là chatbot hỗ trợ của cửa hàng buffet X.\n"
                "Bạn cần hỗ trợ gì ạ? (Có thể hỏi về: đặt bàn, ưu đãi, chi nhánh, giá cả...)"
            )
        elif any(kw in user_input_clean for kw in ["đặt bàn", "muốn đặt"]):
            response = "👍 Bạn muốn đặt bàn cho bao nhiêu người vậy ạ?"
            state["step"] = 1
        elif any(kw in user_input_clean for kw in ["khuyến mãi", "ưu đãi", "giảm giá"]):
            response = (
                "🎉 Hiện chúng mình có ưu đãi giảm 20% cho nhóm từ 4 người trở lên nhân dịp lễ 30/4!\n"
                "Chương trình áp dụng đến hết tuần này nha!"
            )
            state = reset_state()
        elif any(kw in user_input_clean for kw in ["chi nhánh", "ở đâu", "địa chỉ"]):
            response = (
                "📍 Hiện tại Buffet X có 1 chi nhánh tại:\n"
                "315 Đào Duy Từ, Phường 5, Quận 10, TP.HCM"
            )
            state = reset_state()
        elif any(kw in user_input_clean for kw in ["giá", "bao nhiêu tiền", "vé"]):
            response = (
                "💸 Giá vé buffet X là:\n"
                "- Người lớn: 199.000đ\n"
                "- Trẻ em: 119.000đ\n"
                "(Chưa bao gồm VAT)"
            )
            state = reset_state()
        elif any(kw in user_input_clean for kw in ["cảm ơn", "thank"]):
            response = "🌸 Cảm ơn bạn đã liên hệ! Chúc bạn một ngày vui vẻ!"
            state = reset_state()
        else:
            response = (
                "🤖 Xin lỗi, mình chưa hiểu yêu cầu của bạn.\n"
                "Bạn có thể hỏi về đặt bàn, ưu đãi, chi nhánh, giá cả, v.v."
            )

    elif step == 1:
        state["people"] = user_input
        response = "📅 Bạn muốn đặt vào **ngày và giờ** nào ạ? (VD: 12/05 lúc 19h)"
        state["step"] = 2

    elif step == 2:
        state["time"] = user_input
        response = "📝 Quý khách vui lòng cho mình biết tên được không ạ?"
        state["step"] = 3

    elif step == 3:
        state["name"] = user_input
        response = "📞 Cuối cùng, quý khách cho mình xin số điện thoại để xác nhận nhé?"
        state["step"] = 4

    elif step == 4:
        state["phone"] = user_input
        response = (
            f"✅ Vâng thưa quý khách **{state['name']}**, bạn đã đặt bàn thành công:\n"
            f"👥 Số người: {state['people']}\n"
            f"📅 Thời gian: {state['time']}\n"
            f"📞 SĐT: {state['phone']}\n\n"
            "Chúng mình sẽ liên hệ xác nhận sớm nhất nhé! 🎉"
        )
        state = reset_state()

    return response, state


In [39]:
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 |Chatbot hỗ trợ|  - Buffet X")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(placeholder="Nhập lời nhắn của bạn...", show_label=False)
    state = gr.State(reset_state())

    def on_submit(message, chat_history, state_data):
        reply, new_state = chatbot_step(message, state_data)
        chat_history.append((message, reply))
        return chat_history, new_state

    user_input.submit(on_submit, [user_input, chatbot, state], [chatbot, state])


<ipython-input-39-5acefe9b036b>:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [40]:

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://07595ccf068d9f339d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
